In [ ]:
import json
from isce2_topsapp.packaging import package_gunw_product
from shapely.geometry import shape
from pathlib import Path
from isce2_topsapp.topsapp_params import topsappParams
from shapely.geometry import shape
from isce2_topsapp.solid_earth_tides import update_gunw_with_solid_earth_tide
from isce2_topsapp.__main__ import update_gunw_internal_version_attribute
from isce2_topsapp.delivery_prep import prepare_for_delivery

In [ ]:
loc_data = json.load(open('loc_data.json'))

In [ ]:
loc_data

In [ ]:
ref_properties = loc_data["reference_properties"]
sec_properties = loc_data["secondary_properties"]
extent = loc_data["extent"]
product_geometry_wkt = shape(loc_data['gunw_geo']).wkt
loc_data['gunw_geo'] = shape(loc_data['gunw_geo'])
topsapp_params_obj = topsappParams(**loc_data['tops_app_params'])

In [ ]:
with open('additional_2d_layers.txt') as f:
    additional_2d_layers = f.readlines()
additional_2d_layers_for_packaging = [l.strip() for l in additional_2d_layers]
additional_2d_layers_for_packaging

In [ ]:
additional_attributes_for_packaging = json.load(open('additional_attributes_for_packaging.json'))
additional_attributes_for_packaging

In [ ]:
ref_properties = loc_data["reference_properties"]
sec_properties = loc_data["secondary_properties"]
extent = loc_data["extent"]
product_geometry_wkt = shape(loc_data['gunw_geo']).wkt
cmd_line_str = loc_data['cmd_line_str']

In [ ]:
nc_path = package_gunw_product(
    isce_data_directory=Path.cwd(),
    reference_properties=ref_properties,
    secondary_properties=sec_properties,
    extent=extent,
    additional_2d_layers=additional_2d_layers_for_packaging,
    additional_attributes=additional_attributes_for_packaging,
    standard_product=topsapp_params_obj.is_standard_gunw_product(),
    cmd_line_str=cmd_line_str,
    product_geometry_wkt=product_geometry_wkt,
    topaspp_params=topsapp_params_obj.model_dump()
)

In [ ]:
nc_path = update_gunw_with_solid_earth_tide(nc_path, "reference", loc_data['reference_orbits'])
nc_path = update_gunw_with_solid_earth_tide(nc_path, "secondary", loc_data['secondary_orbits'])

In [ ]:
update_gunw_internal_version_attribute(nc_path, new_version="1c")

In [ ]:
final_prod_directory = prepare_for_delivery(nc_path, loc_data)